# Practica Final

Aldao Amoedo, Héctor

Cabaleiro Pintos, Laura

Cotardo Valcárcel, Donato José

Romero Conde, José

---

##   _Librerias_

In [1]:
import Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Statistics")
Pkg.add("StatsBase")
Pkg.add("Random")
Pkg.add("ScikitLearn")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [2]:
using CSV
using DataFrames
using Plots
using Statistics
using Random
using ScikitLearn
using ScikitLearn.CrossValidation: cross_val_score
using ScikitLearn.Pipelines: Pipeline, named_steps, FeatureUnion
using ScikitLearn.GridSearch: GridSearchCV 
@sk_import decomposition: (PCA, FastICA)
@sk_import discriminant_analysis: LinearDiscriminantAnalysis
@sk_import feature_selection: (SelectKBest, f_classif, mutual_info_classif, RFE)
@sk_import impute: SimpleImputer
@sk_import linear_model: LogisticRegression
@sk_import manifold: (LocallyLinearEmbedding, Isomap)
@sk_import neighbors: KNeighborsClassifier
@sk_import neural_network: MLPClassifier
@sk_import preprocessing: MinMaxScaler
@sk_import svm: SVC

PyObject <class 'sklearn.svm._classes.SVC'>

## Preparación de los datos

El punto 1, está completado, aunque creo que el 1.3, donde preparamos los datos transformandolos y rellanando los nulos está mal.

### 1. Cargar los datos y descripción

In [3]:
df = CSV.read("Datos_Practica_Evaluacion_1.csv", DataFrame)

num_instancias, num_variables = size(df)
num_individuos = length(unique(df[:, 1]))
num_clases_salida = length(unique(df[:, end]))

println("Número de variables: $num_variables")
println("Número de instancias: $num_instancias")
println("Número de individuos: $num_individuos")
println("Número de clases de salida: $num_clases_salida")

Número de variables: 563
Número de instancias: 10299
Número de individuos: 30
Número de clases de salida: 6


Los datos dados ya están cargados, y como podemos observar tienen:
* 563 Variables
* 10299 Instancias
* 30 Individuos
* 6 Clases de salida

### 2. Calcular porcentaje de nulos

In [4]:
num_nulos_totales = 0

for col ∈ names(df)
    num_nulos = count(ismissing, df[:, col])
    num_nulos_totales += num_nulos
    porcentaje_nulos = (num_nulos / num_instancias) * 100
end

porcentaje_nulos_totales = (num_nulos_totales / (num_instancias * num_variables)) * 100
println("Porcentaje total de nulos en el conjunto: $porcentaje_nulos_totales%")  

Porcentaje total de nulos en el conjunto: 0.004656507546905259%


### 3. Preparar los datos para las técnicas de clasificación

Para rellenar valores faltantes tenemos que hacernos una idea de que tipo de datos encontraremos.

In [5]:
for col ∈ names(df)
    if eltype(df[!, col]) == Union{Missing, Float64}
        df[ismissing.(df[!, col]), col] .= mean(skipmissing(df[!, col]))
        df[!, col] = Float64.(df[!, col]) 
    end
end

println("Valores nulos rellenados.")

Valores nulos rellenados.


In [6]:
Set([eltype(df[!,col]) for col in names(df)])  # ya no hay missing

Set{DataType} with 3 elements:
  String31
  Int64
  Float64

### 4. Segmentar el 10% de los datos usando HoldOut

In [7]:
Random.seed!(172)

holdout_individuos = shuffle(unique(df[:, :subject]))[1:Int(round(0.1 * length(unique(df[:, :subject]))))]  
holdout_df = filter(fila -> fila.subject in holdout_individuos, df)
train_df = filter(fila -> !(fila.subject in holdout_individuos), df)

println("Individuos en el holdout: ", holdout_individuos)
println("Tamaño del conjunto de entrenamiento: $(size(train_df)[1])")
println("Tamaño del conjunto de holdout: $(size(holdout_df)[1])")

holdout_subjects = unique(holdout_df[:, :subject])
train_subjects = unique(train_df[:, :subject]);

Individuos en el holdout: [4, 12, 29]
Tamaño del conjunto de entrenamiento: 9318
Tamaño del conjunto de holdout: 981


In [8]:
# separacion X Y
X_train = Array(train_df[!,1:end-1])
Y_train = Vector(train_df[!,end])
X_test = Array(holdout_df[!,1:end-1])
Y_test = Vector(holdout_df[!,end]);
# escalado
X_train = fit_transform!(MinMaxScaler(), X_train);
X_test = fit_transform!(MinMaxScaler(), X_test);

In [9]:
function plot_transformed_data(name::String, reductor, X, y)

    X_reducida = fit_transform!(reducer, X, y)
    plot = scatter(X_reduced[:, 1], X_reduced[:, 2], group=y, legend=:topright, title=name,
                xlabel="Componente 1", ylabel="Componente 2", markersize=5)
    return plot
end

plot_transformed_data (generic function with 1 method)

In [10]:
resultados = DataFrame(filtrado = String[], reduccion = String[], Accuracy = Float64[])
plot_distribucion = @layout [a b c; d e]

filtrado = Dict(
   #"nada" => "passthrough",
   "anova" => SelectKBest(score_func=f_classif),
   "mi" => SelectKBest(score_func=mutual_info_classif),
   "rfe" => RFE(LogisticRegression(max_iter=500),n_features_to_select=0.5)
 )

reduccion = Dict(
   #"nada" => "passthrough",
   "pca" => PCA(),
   "lda" => LinearDiscriminantAnalysis(),
   "ica" => FastICA(),
   "isomap" => Isomap(n_neighbors=25),
   "lle" => LocallyLinearEmbedding(),
 )

clasificacion = Dict(
    "mlp" => [MLPClassifier(max_iter=700), Dict(:classifier__hidden_layer_sizes => [[50], [100], [100, 50]])],
    "knn" => [KNeighborsClassifier(), Dict(:classifier__n_neighbors =>[1, 10, 20])],
    "svm" => [SVC(), Dict(:classifier__C =>[0.1, 1, 10])]
)
  
for (nombreFiltro, filtro) in filtrado
    for (nombreReduccion, reduccion) in reduccion
        for (nombreClasificador, valor) in clasificacion

            clasificador = valor[1]
            parametros_local = valor[2]
            modelo = Pipeline([
                ("filtro", filtro),
                ("reduccion", reduccion),
                ("classifier", clasificador) 
            ])

            busqueda = GridSearchCV(modelo, parametros_local; refit=true, cv = 2, n_jobs=3)
            fit!(busqueda, X_train, Y_train)
            mejorModelo = busqueda.best_estimator_
            mejoresParametros = busqueda.best_params_
            Y_pred = predict(mejorModelo, X_test)
            accuracy = sum(Y_pred .== Y_test) / length(Y_test)
            push!(resultados, (nombreFiltro, nombreReduccion,accuracy))
            println("Filtrado: $nombreFiltro,\nReducción: $nombreReduccion,\nClasificador: $nombreClasificador,\nparámetros: $(mejoresParametros),\nPrecisión: $accuracy\n")
        
        end
    end
end

LoadError: UndefVarError: `bb` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

# hasta aqui

In [ ]:




param_grid = Dict(
    #"filtrado__pca__n_components" => [ 50, 100], 
    "filtrado"
    "knn__n_neighbors" => [ 100, 200], 
)


pipe = Pipeline([
        ("eliminarNulos", SimpleImputer())
        ("escalado", MinMaxScaler())
        ("filtrado", filtrado)
        #("reduccionDimensionalidad", )
        ("knn",KNeighborsClassifier())
        ])

grid_search = GridSearchCV(pipe, param_grid, cv=2)

fit!(grid_search, X_train, Y_train)

println("Mejor modelo: ", grid_search.best_estimator_)
println("Mejores hiperparámetros: ", grid_search.best_params_)

In [ ]:
pipe = Pipeline([
        ("eliminarNulos", SimpleImputer())
        ("escalado", MinMaxScaler())
        ("filtrado", FeatureUnion([
            ("pca",PCA()),
            ("anova",SelectKBest(score_func=f_classif)),
            ("mi",SelectKBest(score_func=mutual_info_classif)),
            ("rfe",RFE(LogisticRegression(),n_features_to_select=0.5)),
        ]))
        ("knn",KNeighborsClassifier())
        ])

param_grid = Dict(
    "filtrado__pca__n_components" => [ 50, 100], 
    "knn__n_neighbors" => [ 100, 200], 
)

grid_search = GridSearchCV(pipe, param_grid, cv=2)

fit!(grid_search, X_train, Y_train)

println("Mejor modelo: ", grid_search.best_estimator_)
println("Mejores hiperparámetros: ", grid_search.best_params_)

In [ ]:
pca = PCA(n_components=2)

selection = SelectKBest(k=1)

# Build estimator from PCA and Univariate selection:

combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])

# Use combined features to transform dataset:
X_features = transform(fit!(combined_features, X, y), X)

svm = SVC(kernel="linear")

# Do grid search over k, n_components and C:

pipeline = Pipeline([("features", combined_features), ("svm", svm)])

param_grid = Dict(:features__pca__n_components=>[1, 2, 3],
                  :features__univ_select__k=>[1, 2],
                  :svm__C=>[0.1, 1, 10])

grid_search = GridSearchCV(pipeline, param_grid; verbose=10, refit=true)

fit!(grid_search, X, y)

print(grid_search.best_estimator_)

### 6. Normalizar usando MinMaxScaler

---

## Creación de los modelos básicos

### 7. Filtrado

#### 7.1 ANOVA

#### 7.2 Mutual Information

#### 7.3 RFE con el método de LogisticRegression con una eliminación del 50% de las variables en cada pasada.

### 8. Reducción dimensionalidad

#### 8.1 PCA

#### 8.2 LDA

#### 8.3 ICA

#### 8.4 Isomap

#### 8.5 LLE

### 9. Clasificadores

#### 9.1 MLP con al menos las siguientes arquitecturas: [50], [100] [100, 50]

#### 9.2 KNN con valores de vecindario entre 1, 10 y 20

#### 9.3 SVM con el parámetro C con valores 0.1, 0.5 y 1.0

---

## Creación de los modelos ensemble

---

## Conclusiones